Load API Keys

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

Load Contents from HLB Fixed Deposite Pages

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

Split the Loaded Data into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)
texts

Initialize Pinecone

In [ ]:
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-conversation-chain"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

Create Embeddings and Vectors to Insert to Pinecone Database

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [ ]:
retriever = vector.as_retriever(search_kwargs={'k': 15})
retriever

Creating Prompt Template

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: {question}
  By only basing on this context: {context}
  Don't need to refer to chat history as long as you have enough information from the context.
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the input, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input requires you to do something creative such as writing a song or making a joke.
"""
prompt =PromptTemplate.from_template(prompt_template)

Creating Memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(llm=llm, memory_key="chat_history", output_key="answer", k=3, return_messages=True)

Creating Chain

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=retriever,
  memory=memory,
  verbose=True,
  combine_docs_chain_kwargs={"prompt": prompt}
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain(question)
    print(f'{cb}\n')
    print(f'\nAnswer: {result["answer"]}')


Running the Chain

In [ ]:
ask_question("How many types of fixed deposit are there?")

In [ ]:
ask_question("what are the interest rates for 1st one?")

In [ ]:
ask_question("What is e-fixed deposit?")

In [ ]:
ask_question("What are the pros and cons of fixed deposit?")

In [ ]:
ask_question("Who is Jay Z?")

In [ ]:
ask_question("How many types of fixed deposit CIMB provide?")

In [ ]:
ask_question("What are the interest rates for Fixed Deposit?")

In [ ]:
ask_question("Can you find out by browsing the web?")

In [ ]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 36 months and the interest rate is 5%. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest?")

In [ ]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.")

In [ ]:
ask_question("Can a non-resident of Malaysia invest in CIMB SGD fixed deposit?")

In [ ]:
ask_question("Who is the CEO of CIMB bank?")

In [ ]:
ask_question("What is CIMB net profile for the last financial year?")

In [ ]:
ask_question("What if Jay Z from United States wants to invest in SGD fixed deposit. Is that ok ?")

In [ ]:
ask_question("What if Michael Jackson wants to invest in SGD Fixed Deposit. Is that ok?")

In [ ]:
ask_question("Write a poem for HLB fixed deposit starting with Roses are Red, Violets are blue")

In [ ]:
ask_question("how many types of saving accounts are there in a general bank except HLB?")

In [ ]:
ask_question("What is fixed deposit?")

In [ ]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")

In [ ]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")

In [ ]:
ask_question("What are the interest rates for standard Fixed Deposit?")

In [ ]:
ask_question("Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw  at the end of the term.")